In [1]:
#Finding stay points from a user trajectory data and calculate transition matrices.
#Stay points are goverened by two parameters, time threshold and distance threshold.
#If user has spend more than threshold time(10 mins) at the same geographical location(threshold distance 50m),
# then this location is users stay point.
#This can be users home, work, a bus station or a restaurant. 
%reset

#load all the files for a user
import random
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import math 
import os
import errno
import matplotlib.patches as patches
from copy import deepcopy
from scipy.spatial.distance import cdist
from matplotlib.patches import Ellipse
import operator
import pdb
import calendar
#%matplotlib nbagg

pd.options.mode.chained_assignment = None

#-----------------------------------LOAD USER FILES-----------------------------------------------------
def load_user_file():
    global combined_df
    #Load file names for user. Change here for different user replace 000 or even path as required
    filenames = glob.glob(file_source_raw)

    #Read the files
    list_of_dfs = [pd.read_csv(filename, skiprows=6, header = None) for filename in filenames]

    #put the data from list into one dataframe
    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    #rename columns
    combined_df.columns = ['Latitude', 'Longitude', '0', 'Altitude', 'NumDays', 'Date', 'Time']
    combined_df['ClusterId'] = -1
    combined_df['ClusterMeanLat'] = -1
    combined_df['ClusterMeanLon'] = -1
    combined_df['StayPoint'] = -1
    combined_df['SignificantPlace'] = -1

    #add timestamp index
    combined_df["Timestamp"] = combined_df["Date"].map(str) + " " + combined_df["Time"]
    combined_df.Timestamp = pd.to_datetime(combined_df.Timestamp)
    combined_df.index = pd.to_datetime(combined_df.Timestamp)
    
    #add weekday number as column
    combined_df['Weekday'] = combined_df['Timestamp'].dt.weekday.map(str) + combined_df['Timestamp'].dt.weekday_name

    #combined_df.size
    #combined_df.head()

#-----------------------------------SAMPLE DATA PER MINUTE-----------------------------------------------------

def resample_select_data():
    global combined_df
    global sampled_df
    
    #Resample the data with every one minutes. Remove this if you like to process the entire file.
    #Note, it could take some time to run the further sections of the file size is very large.
    sampled_df = combined_df.resample('1T').mean()
    sampled_df["Timestamp"] = sampled_df.index
    sampled_df = sampled_df.dropna()
    
    #Select the range of data you want to proceed with
    if fltr_data_date_rng == "YES":
        sampled_df = sampled_df[(sampled_df["Timestamp"] >= from_date) & (sampled_df["Timestamp"] <= to_date)]

#-----------------------------------VISUALIZE RAW DATA--------------------------------------------------------

def visualize_raw_data():
    
    if fltr_data_date_rng == "YES":
        combined_c_df = combined_df[(combined_df["Timestamp"] >= from_date) & (combined_df["Timestamp"] <= to_date)]
    else:
        combined_c_df = combined_df
        
    #Plot 1------------------
    #plot hourly vs weekly data of the user
    #Resample data on hourly basis
    hour_sampled_df = combined_c_df.resample('H').last()

    #drop column timestamp to avoid confusion as index is also timestamp
    hour_sampled_df = hour_sampled_df.drop(['Timestamp'], axis=1)

    #extract time from timestamp and add it as a column
    hour_sampled_df['TimeSampled'] = hour_sampled_df.index.time

    #form a pivot table counting number of latitudes for each weekday for each hour
    pivot_df = hour_sampled_df.pivot_table(values='Latitude', index='Weekday',columns='TimeSampled',aggfunc=len)

    #draw a plot to visualize hourly trend per weekday for the user
    fig, ax = plt.subplots(figsize=(20,5))  
    sns.heatmap(pivot_df, cmap='BuGn', ax=ax)
    plt.title('Weekly - Hourly Plot')
    
    #Plot 2------------------
    #visualize counts of data for each month-year
    count_df = pd.DataFrame()
    count_df['count'] = combined_c_df.groupby(combined_c_df.index.year * 100 + combined_c_df.index.month).size() 
    count_df.plot(kind='bar', title ="V comp",figsize=(5,5),legend=True, fontsize=12, color = 'b')
    plt.title('MonthYear Distribution Plot')
    
    #Plot 3------------------
    #plot to visualize the location trend of the user
    #Sample data hourly and take mean, drop nans, reset index
    xy_df = combined_c_df.resample('H').mean()
    xy_df = xy_df.dropna()
    xy_df['Timestamp'] = xy_df.index
    xy_df = xy_df.reset_index(drop=True)

    #assign the first lat and log as the base for the plot i.e. origin
    origin_lat = math.radians(xy_df["Latitude"][0])
    origin_lon = math.radians(xy_df["Longitude"][0])
    
    #convert each lat and lon into x and y for the plot w.r.t origin
    EARTH_RAD = 6378100
    xy_df['X'] = 0.0
    xy_df['Y'] = 0.0
    for i in range(0, len(xy_df)):
        x = 0
        y = 0
        current_lat = math.radians(xy_df["Latitude"][i])
        current_lon = math.radians(xy_df["Longitude"][i])
        x = ((math.cos(current_lat) + math.cos(origin_lat))/2) * EARTH_RAD * (current_lon - origin_lon) * math.pi / 180
        y = (current_lat - origin_lat)* math.pi/180 * EARTH_RAD
        xy_df.at[i, 'X'] = x
        xy_df.at[i, 'Y'] = y
    
    xy_copy_df = xy_df[['X', 'Y', 'Timestamp']].copy()
    xy_copy_df['Hour'] = xy_copy_df.Timestamp.dt.hour
    del xy_copy_df['Timestamp']
    fg = sns.FacetGrid(data=xy_copy_df, hue='Hour',  size = 10, aspect = 2)
    fg.map(plt.scatter, 'X', 'Y', edgecolor="w", s=400).add_legend()
    plt.title('User Movement Trend Plot')
    plt.xlabel('X')
    plt.ylabel('Y')
    xticks = np.arange(-1000,1000,50)
    yticks = np.arange(-1000,1000,50)
    fg.set(xticks=xticks, yticks=yticks)
    plt.savefig(dest_file1)
    


#-----------------------------------METER DISTANCE BETWEEN TWO COORDINATES---------------------------------------

#Find distance between two lan:lon points in meters
def meters(lat1, lon1, lat2, lon2):  
    R = 6378.137 # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
    d = R * c
    return d * 1000 # meters

#This function is to cluster the points together if thier distance is less than 50 meters.
#If the cluster has a total duration of 10 minutes or greater, then add it to a stay point

#-----------------------------------CLUSTER POINTS----------------------------------------------------------

def cluster(newlat, newlon, row, count):
    global sampled_df
    
    currcluster = sampled_df['ClusterId'][row-1]
    sampled_df['ClusterId'][row] = -1
    sampled_df['ClusterMeanLat'][row] = -1
    sampled_df['ClusterMeanLon'][row] = -1
    sampled_df['StayPoint'][row] = -1
    sampled_df['SignificantPlace'][row] = -1
    clulat = sampled_df['ClusterMeanLat'][row-1]
    clulon = sampled_df['ClusterMeanLon'][row-1]
    
    #import pdb; pdb.set_trace()
    if meters(clulat, clulon, newlat, newlon)<= 100:
        sampled_df['ClusterId'][row] = currcluster
        sampled_df['ClusterMeanLat'] = sampled_df.groupby('ClusterId')['Latitude'].transform(np.mean)
        sampled_df['ClusterMeanLon'] = sampled_df.groupby('ClusterId')['Longitude'].transform(np.mean)
        count = count + 1
    else:
        
        if count >= 2:
            #import pdb; pdb.set_trace()
            MinClusTime = sampled_df['Timestamp'][row-count]
            MaxClusTime = sampled_df['Timestamp'][row-1]
            k = MaxClusTime - MinClusTime
            l = (k / np.timedelta64(1, 'm')).astype(int)
            
            if (l >= 30):
                sampled_df.loc[ (sampled_df['ClusterId']==currcluster), 'StayPoint'] = 1
        count = 1
        sampled_df['ClusterMeanLat'][row] = sampled_df['Latitude'][row]
        sampled_df['ClusterMeanLon'][row] = sampled_df['Longitude'][row]
        sampled_df['ClusterId'][row] = currcluster + 1
    return count

#-----------------------------------FIND STAYPOINTS USING CLUSTERS---------------------------------------------------

def find_stay_points():
    global sampled_df 
    global staypts_df
    
    #Read the file in an online manner as the points come and assign the points to clusters
    row =1
    count = 1
    sampled_df['ClusterId'][row-1] = 0
    sampled_df['ClusterMeanLat'][row-1] = sampled_df['Latitude'][0]
    sampled_df['ClusterMeanLon'][row-1] = sampled_df['Longitude'][0]
    sampled_df['StayPoint'][row-1] = -1
    sampled_df['SignificantPlace'][row-1] = -1
    while row < len(sampled_df):
        #import pdb; pdb.set_trace()
        count = cluster(sampled_df['Latitude'][row], sampled_df['Longitude'][row], row, count)
        row= row + 1

    #copy the stay points into another dataframe
    staypts_df = sampled_df.loc[sampled_df['StayPoint'] == 1]
    
#-----------------------------------GROUP CLUSTERS ON DIFFERENT DAYS-----------------------------------------------

def group_clusters():
    global staypts_df
    
    #this fucntion groups the clusters together from different days 
    #Copy the stay points dataframe into another dataframe and remove duplicates
    staypts_df1 = staypts_df[['ClusterId', 'ClusterMeanLat', 'ClusterMeanLon']].copy()
    staypts_df1 = staypts_df1.drop_duplicates(subset=['ClusterId', 'ClusterMeanLat', 'ClusterMeanLon'])

    staypts_df1 = staypts_df1.sort_values(['ClusterMeanLat', 'ClusterMeanLon'])

    row = 1
    for i in range(0, len(staypts_df1)):
        for j in range(i+1, len(staypts_df1)):
            #import pdb; pdb.set_trace()
        
            chk_cluster = staypts_df1['ClusterId'][i]
            chk_clulat = staypts_df1['ClusterMeanLat'][i]
            chk_clulon = staypts_df1['ClusterMeanLon'][i]
            curr_cluster = staypts_df1['ClusterId'][j]
            curr_clulat = staypts_df1['ClusterMeanLat'][j]
            curr_clulon = staypts_df1['ClusterMeanLon'][j]
        
            if meters(chk_clulat, chk_clulon, curr_clulat, curr_clulon)<= 100:
                staypts_df.loc[ (staypts_df['ClusterId']==curr_cluster), 'ClusterId'] = chk_cluster
                staypts_df['ClusterMeanLat'] = staypts_df.groupby('ClusterId')['ClusterMeanLat'].transform(np.mean)
                staypts_df['ClusterMeanLon'] = staypts_df.groupby('ClusterId')['ClusterMeanLon'].transform(np.mean)
            else:
                break

#-----------------------------------SAVE STAYPOINTS AS FILE-----------------------------------------------------

def save_file_stay_points():
    #Save the file to a location for further analysis
    staypts_df.to_csv(dest_file_staypoints, sep='\t', encoding='utf-8')

#------------------------------------VISUALIZE STAYPOINTS-------------------------------------------------------

def visualize_stay_points():
    global staypts_df
    #Plot the significant places

#     #load the staypoints file
#     staypts_df = pd.read_csv(dest_file_staypoints, sep='\t')

    #make index as datetime
    staypts_df.index = pd.to_datetime(staypts_df.Timestamp)

    #assign the first lat and log as the base for the plot i.e. origin
    origin_lat = math.radians(combined_df["Latitude"][0])
    origin_lon = math.radians(combined_df["Longitude"][0])
    staypts_df['Timestamp'] = staypts_df.index
    staypts_df = staypts_df.reset_index(drop=True)

    #convert each lat and lon into x and y for the plot w.r.t origin
    EARTH_RAD = 6378100
    staypts_df['X'] = 0.0
    staypts_df['Y'] = 0.0
    for i in range(0, len(staypts_df)):
        x = 0
        y = 0
        current_lat = math.radians(staypts_df["Latitude"][i])
        current_lon = math.radians(staypts_df["Longitude"][i])
        x = ((math.cos(current_lat) + math.cos(origin_lat))/2) * EARTH_RAD * (current_lon - origin_lon) * math.pi / 180
        y = (current_lat - origin_lat)* math.pi/180 * EARTH_RAD
        staypts_df.at[i, 'X'] = x
        staypts_df.at[i, 'Y'] = y

    #plot the x and y's
    staypts_copy_df = staypts_df[['X', 'Y', 'Timestamp']].copy()
    staypts_copy_df['Hour'] = staypts_copy_df.Timestamp.dt.hour
    del staypts_copy_df['Timestamp']
    #xy_test_df.head()
    fg = sns.FacetGrid(data=staypts_copy_df, hue='Hour',   size = 10, aspect = 2)
    fg.map(plt.scatter, 'X', 'Y', edgecolor="w", s = 400).add_legend()
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title("User Staypoints Trend Plot")
    xticks = np.arange(-1000,1000,50)
    yticks = np.arange(-1000,1000,50)
    fg.set(xticks=xticks, yticks=yticks)
    plt.savefig(dest_file2)
    
    staypts_df.index = pd.to_datetime(staypts_df.Timestamp)
#-----------------------------------CALCULATE HOURLY WEIGHTS FOR STAYPOINTS--------------------------------------
def cal_hourly_cluster_weight():
    global staypts_df
    global cluster_hourly_df    

    for i in range(0, 24):
        cluster_hourly_df['Date'] = 0
        cluster_hourly_df['ClusterId'] = 0
        cluster_hourly_df['AvgLat'] = 0
        cluster_hourly_df['AvgLon'] = 0
        cluster_hourly_df[i] = 0

    last_hour = staypts_df['Timestamp'][0].hour
    last_clusid = staypts_df['ClusterId'][0]
    curr_count = 0
    j = 0
    
    for i in range(0, len(staypts_df)):

        if (i == len(staypts_df)-1):
            
            k = staypts_df['Timestamp'][i] - staypts_df['Timestamp'][i-curr_count+1]
            l = (k / np.timedelta64(1, 'm')).astype(int)
            
            date_read = staypts_df.index[i].date()
            cluster_id = staypts_df['ClusterId'][i]
            ClusterMeanLat = staypts_df['ClusterMeanLat'][i]
            ClusterMeanLon = staypts_df['ClusterMeanLon'][i]
            col_name = staypts_df.index[i].hour

            cluster_hourly_df.loc[j,'AvgLat'] = ClusterMeanLat
            cluster_hourly_df.loc[j,'AvgLon'] = ClusterMeanLon
            cluster_hourly_df.loc[j,'Date'] = date_read
            cluster_hourly_df.loc[j,'ClusterId'] = cluster_id
            cluster_hourly_df.loc[j, col_name] = round((l)/60,4)
            
        if (staypts_df['Timestamp'][i].hour != last_hour) | (staypts_df['ClusterId'][i] != last_clusid):
            #import pdb; pdb.set_trace()

            if (curr_count == 1) & (staypts_df['Timestamp'][i].hour != last_hour):
                k = ((staypts_df['Timestamp'][i-1] + pd.Timedelta(hours=1) - 
                      pd.Timedelta(minutes=staypts_df['Timestamp'][i-1].minute)) - 
                     staypts_df['Timestamp'][i-1])
            else:
                k = staypts_df['Timestamp'][i-1] - staypts_df['Timestamp'][i-curr_count]

            l = (k / np.timedelta64(1, 'm')).astype(int)
            date_read = staypts_df.index[i-1].date()
            cluster_id = staypts_df['ClusterId'][i-1]
            ClusterMeanLat = staypts_df['ClusterMeanLat'][i-1]
            ClusterMeanLon = staypts_df['ClusterMeanLon'][i-1]
            col_name = staypts_df.index[i-1].hour

            cluster_hourly_df.loc[j, 'AvgLat'] = ClusterMeanLat
            cluster_hourly_df.loc[j, 'AvgLon'] = ClusterMeanLon
            cluster_hourly_df.loc[j, 'Date'] = date_read
            cluster_hourly_df.loc[j, 'ClusterId'] = cluster_id
            cluster_hourly_df.loc[j, col_name] = round((l)/60,4)
            j = j + 1
            curr_count = 1

            if (staypts_df['Timestamp'][i].hour != last_hour):
                last_hour = staypts_df['Timestamp'][i].hour
            if (staypts_df['ClusterId'][i] != last_clusid):
                last_clusid = staypts_df['ClusterId'][i]
        else:
            curr_count = curr_count + 1

    cluster_hourly_df = cluster_hourly_df.fillna(0)
    cluster_hourly_df = cluster_hourly_df.groupby(['Date', 'ClusterId', 'AvgLat', 'AvgLon']).sum()
    cluster_hourly_df = cluster_hourly_df.reset_index(level=[0,1,2,3])

#--------------------------------VISUALIZE HOURLY CLUSTER WEIGHT-------------------------------------------------
def visualize_hourly_cluster_weight():
    global staypts_df
    
    #create a color dictionary for each cluster for the plot
    dicts = {}
    clu_list = []
    clu_list = staypts_df['ClusterId'].unique()
    #r = lambda: random.randint(0,255)
    colors = sns.color_palette("Paired", len(clu_list))
    
    for i in range(0, len(clu_list)):
        dicts[clu_list[i]] = (colors[i])
        #dicts[clu_list[i]] = ('#%02X%02X%02X' % (r(),r(),r()))
        
    #create a new graph where we will later add rectangles for each hour:cluster
    fig2 = plt.figure(figsize=(15,15))
    ax1 = fig2.add_subplot(111, aspect='equal')

    #get all the dates for y axis
    date_list = staypts_df['Timestamp'].dt.date.unique()
    y = range(0, len(date_list))
    def_yticks = date_list
    plt.yticks(y, def_yticks)
    
    #set the x axis limit from 0-24 hours of a day, y axis with dates
    limsx = (0, 24)
    limsy = (0, len(date_list))

    date_counter = 0
    last_date = staypts_df['Timestamp'][0].date()
    last_hour = staypts_df['Timestamp'][0].hour
    last_clusid = staypts_df['ClusterId'][0]
    curr_count = 0
    j = 0

    for i in range(0, 24):
        ax1.axvline(x= i, linewidth=1, color='r')

    for i in range(0, len(staypts_df)):
        #import pdb; pdb.set_trace()

        #plot a rectangle if the hour or clusterid or date has changed
        if ((staypts_df['Timestamp'][i].hour != last_hour) | (staypts_df['ClusterId'][i] != last_clusid)
           | (last_date != staypts_df['Timestamp'][i].date())):

            if (curr_count == 1) & (staypts_df['Timestamp'][i].hour != last_hour):
                a = staypts_df['Timestamp'][i-curr_count].hour + 1
            else:
                a = staypts_df['Timestamp'][i-curr_count].hour + staypts_df['Timestamp'][i-curr_count].minute/60

            b = staypts_df['Timestamp'][i-1].hour + staypts_df['Timestamp'][i-1].minute/60

            width = b - a
            height = 1
            col_id = dicts.get(staypts_df['ClusterId'][i-1])
            ax1.add_patch(patches.Rectangle((a, date_counter), width, height, color=col_id, label=staypts_df['ClusterId'][i-1]))

            curr_count = 1

            if (staypts_df['Timestamp'][i].hour != last_hour):
                last_hour = staypts_df['Timestamp'][i].hour
            if (staypts_df['ClusterId'][i] != last_clusid):
                last_clusid = staypts_df['ClusterId'][i]
            if (last_date != staypts_df['Timestamp'][i].date()):
                date_counter = date_counter + 1
                last_date = staypts_df['Timestamp'][i].date()
                ax1.axhline(y= date_counter, linewidth=1, color='r')

        else:
            curr_count = curr_count + 1
            
    handles, labels = ax1.get_legend_handles_labels()
    handle_list, label_list = [], []
    for handle, label in zip(handles, labels):
        if label not in label_list:
            handle_list.append(handle)
            label_list.append(label)
    plt.legend(handle_list, label_list)

    plt.xlim(limsx)
    plt.ylim(limsy)
    plt.savefig(dest_file3)


#---------------------------------------------- S  T  A  R  T  -------------------------------------------------
#Start of the program
                                                                                                                                                                                                                                                                                                                                                                                                        


#Load file names for users
for i in range(0, 51):
    #global dataframes used
    combined_df = pd.DataFrame()
    sampled_df = pd.DataFrame()
    staypts_df = pd.DataFrame()
    cluster_hourly_df = pd.DataFrame()

    user = "%03d"%i
    path = "/home/shashank/Documents/location/Geolife Trajectories 1.3/Data/" + str(user) + "/Trajectory/20*.plt"
    
    filenames = glob.glob(path)

    #Read the files
    list_of_dfs = [pd.read_csv(filename, skiprows=6, header = None) for filename in filenames]

    #put the data from list into one dataframe
    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    #rename columns
    combined_df.columns = ['Latitude', 'Longitude', '0', 'Altitude', 'NumDays', 'Date', 'Time']

    #add filename as column to identify each row with its source
    for filename in filenames:
        combined_df['Filename'] = filename[-18:]

    #add timestamp as columns and then make timestamp as index
    combined_df["Timestamp"] = combined_df["Date"].map(str) + " " + combined_df["Time"]
    combined_df.Timestamp = pd.to_datetime(combined_df.Timestamp)
    combined_df.index = pd.to_datetime(combined_df.Timestamp)
    
    test = pd.DataFrame()
    test['count'] = combined_df.groupby(combined_df.index.year * 100 + combined_df.index.month).size() 
    test['MonthYear'] = test.index
    mnth_yr = test['MonthYear'].loc[test['count'] == max(test['count'])]
    mnth_yr = mnth_yr.iloc[0]
    mnth_yr = str(mnth_yr)
    yr = mnth_yr[:4]
    mnth = mnth_yr[4:]
    mnth = int(mnth)
    yr = int(yr)
    end_date = calendar.monthrange(yr, mnth)[1]

    #source, destination path and filter till date(till when the data needs to be considered). Change according to your needs
    from_date = str(yr) + "-" + str(mnth) + "-01 00:00:00"
    to_date = str(yr) + "-" + str(mnth) + "-" + str(end_date) + " 23:59:00"
    fltr_data_date_rng = "YES"   

    
    file_source_raw = "/home/shashank/Documents/location/Geolife Trajectories 1.3/Data/" + str(user) + "/Trajectory/20*.plt" 
    destination = "/home/shashank/Documents/location/Data/User stay points(month-maxdata, 100m 30min)/"
    dest_file1 = destination + str(user) + " YearMonth-" + str(yr) + str(mnth) + " Raw Data.png" 
    dest_file2 = destination + str(user) + " YearMonth-" + str(yr) + str(mnth) + " Staypoints.png" 
    dest_file3 = destination + str(user) + " YearMonth-" + str(yr) + str(mnth) + " Hourly Cluster Weights.png" 

    load_user_file()

    resample_select_data()

    visualize_raw_data()

    find_stay_points()
    if not staypts_df.empty:
        group_clusters()

        visualize_stay_points()
    
        cal_hourly_cluster_weight()

        visualize_hourly_cluster_weight()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


/home/shashank/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
sampled_df